# Bayes Theorem
DA Probability & Statistics • Lesson 3

## Topics to Cover
- Motivating question - Together
- Bayes intuition for the 2 event case - NJ
- Updating priors - Shreyas
    - Loop through various priors to demonstrate how the posterior probabilities will update
    - What is the consequence of updating priors?
    - Controversy around priors being subjective: [Prosecutor's Fallacy](https://towardsdatascience.com/the-prosecutors-fallacy-cb0da4e9c039)
- Add a partitioning event and have the students calculate the posterior based on that - Shreyas
- Bayes proof for the generalized case - NJ
- Generalized case example - Together
    - 

## Motivating Question 🤔
> Tell me something that I thought I could answer using what Ravi taught last lesson but in reality is hard to do without Bayes Theorem

- That should explian why calculating probabilities in one direction is harder than calculating it the other way
- **OR** position it as hypothesis and evidence where only one direction is what is interesting to look at

$$
\begin{align}
P(\text{at least one shipment is air}) &= \frac{3}{4}\\
P(\text{the second shipment is air}) &= \frac{1}{2}
\end{align}
$$

- Posterior Probability: 
- Conditional Probability: 
- Prior Probability: 
- Marginal Probability: 